In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import pandas as pd
import os

# Read individual categories

In [ ]:
path = './output/' 
fn = os.listdir(path)
fn = [f for f in fn if 'min2.csv' in f]

In [ ]:
#'/Users/cemgil/src/OpenMaker/Semantics/output/' /Users/hamzazair/Desktop/HamzazProjects/EU\ Projects/Open\ Maker/openmaker\ github/OpenMaker/
import os

path = './output/' 
fn = os.listdir(path)

DF = []
CATEGORIES = []
for f in fn:
    print(f)
    df = pd.read_csv(path+f)
    DF.append(df)
    #CATEGORIES.append(f.split('_')[1].split('.')[0])
    CATEGORIES.append(f[9:].split('.')[0])

In [ ]:
os.listdir(".")

In [ ]:
for i,x, in enumerate(CATEGORIES):
    print(x,i)

In [ ]:
categories_desired_order = [4,0,3,7,6,9,1,8,2,5]
for i in categories_desired_order:
    print(CATEGORIES[i])

In [ ]:
DF[0].Stem

Retrieve a news item

In [ ]:
urls = ['https://www.un.org/sg/en/content/sg/speeches/2018-03-28/collective-action-improve-un-peacekeeping-operations-remarks',
'https://www.un.org/sg/en/content/sg/speeches/2018-03-26/remembrance-victims-slavery-and-transatlantic-slave-trade-remarks',
'https://www.un.org/sg/en/content/sg/speeches/2018-03-23/turtle-bay-security-roundtable-remarks',
'https://www.un.org/sg/en/content/sg/speeches/2018-03-22/decade-action-water-sustainable-development-remarks',
'https://www.whitehouse.gov/briefings-statements/the-inaugural-address/',	
'http://www.nytimes.com/2009/01/20/us/politics/20text-obama.html',	
'http://www.presidency.ucsb.edu/ws/?pid=25853',	
'http://edition.cnn.com/2001/US/09/11/bush.speech.text/',
'http://www.let.rug.nl/usa/documents/1951-/martin-luther-kings-i-have-a-dream-speech-august-28-1963.php']

import requests
import json

insightIP = 'http://178.62.229.16'
insightPort = '8484'
insightVersion = 'v1.0'
apikey = 'thisisinsightapikey'

# parameters of news recommendation are twitter_ids, crm_ids
"""
twitter_ids = []
crm_ids = [135574293]
insightSetting = insightIP + ':' + insightPort + '/api/' + insightVersion 
request = '/recommendation/news?' + 'twitter_ids=' + str(twitter_ids) + '&' + 'crm_ids=' + str(crm_ids)
"""

TXT = []

for url in urls:
    insightSetting = insightIP + ':' + insightPort + '/api/' + insightVersion
    request = '/text_analytics/url_scraper?api_key='+apikey+'&url='+url
    #res = requests.get(insightSetting + request)
    print(insightSetting + request)
    res = requests.get(insightSetting + request)
    data = res.json()
    TXT.append(data['text'])

In [ ]:
TXT[9]

Process the input set according to the same procedure as the seed texts

In [ ]:
with open("data/stopwords_standard.txt", "r") as f:
    STOP_WORDS_STANDARD = set(f.read().strip().split("\n"))
print(STOP_WORDS_STANDARD)

with open("data/stopwords_openmaker.txt", "r") as f:
    STOP_WORDS_OPENMAKER = set(f.read().strip().split("\n"))
print(STOP_WORDS_OPENMAKER)

# merging the two list together
STOP_WORDS = STOP_WORDS_STANDARD.union(STOP_WORDS_OPENMAKER)
print(STOP_WORDS)

In [ ]:
import nltk
from utils import tokenizer
import nltk
from nltk import FreqDist
from nltk.stem.porter import PorterStemmer
from numpy import log, mean
import json, csv, re
import pprint as pp

In [ ]:
tokens = nltk.word_tokenize(TXT[0])

In [ ]:
token_counts = FreqDist(tokens)
tokenizer.CHARACTERS_TO_SPLIT += '‘'+'’'+'“'+'”'+'.'

In [ ]:
for stopword in STOP_WORDS:
    if stopword in token_counts:
        del token_counts[stopword]
        
for punctuation in tokenizer.CHARACTERS_TO_SPLIT:
    if punctuation in token_counts:
        del token_counts[punctuation]

        
pattern_letters = re.compile('[a-z]')
def has_letters(x):
    return(pattern_letters.search(x) is not None)

reduced = {k:v for k,v in token_counts.items() if has_letters(k)}
print("Reduction due to all number matches: ", len(token_counts) - len(reduced))
token_counts = reduced

In [ ]:
token_counts

In [ ]:
reduced = {k:v for k,v in token_counts.items() if len(k) > 1}
print("Reduction due to single characters: ", len(token_counts) - len(reduced))
token_counts = reduced

In [ ]:
token_counts

# Stemming

In [ ]:
stemmer = PorterStemmer()
input_wset_stems = {k: stemmer.stem(k) for k in token_counts.keys()}
input_wset_stems

Catalog contains all the catagory score tables

In [ ]:
catalog = []
for i in range(len(DF)):
    catalog.append({u[1].Stem: u[1].Score for u in DF[i].iterrows()})


Rate the example text according to the scores:

The scoring just adds up the word scores but this must be improved

In [ ]:
Scores = []
for cat_idx in range(len(fn)):
    print(fn[cat_idx])

    sum_score = 0
    notin_catalog = 0
    for p in token_counts.keys():
        if p in input_wset_stems.keys():
            r = input_wset_stems[p]
        else:
            r = p
        
        #print('.....')
        if p in catalog[cat_idx].keys():
            #sum_score += catalog[cat_idx][p] 
            #print(catalog[cat_idx][p], p,token_counts[p])
            None
        else:
            #print('No Score', p,token_counts[p])
            #notin_catalog+=1
            None
            
        if r in catalog[cat_idx].keys():
            sum_score += catalog[cat_idx][r] 
            print(catalog[cat_idx][r], r)
        else:
            print('No Score', r)
            notin_catalog+=1

    print('Count of Terms not found in the catalog:', notin_catalog)   
    print('Score = ', sum_score)
    print(CATEGORIES[cat_idx])
    print('*'*30)
    
    Scores.append(sum_score)
    

Visualize the score of the txt for each category

In [ ]:
for i in range(len(CATEGORIES)):
    print(CATEGORIES[i], Scores[i])
print('-')
for i in categories_desired_order:
    print(CATEGORIES[i], Scores[i])


# Scoring in a nutshell

In [ ]:
urls = ['https://www.un.org/sg/en/content/sg/speeches/2018-03-28/collective-action-improve-un-peacekeeping-operations-remarks',
'https://www.un.org/sg/en/content/sg/speeches/2018-03-26/remembrance-victims-slavery-and-transatlantic-slave-trade-remarks',
'http://www.let.rug.nl/usa/documents/1951-/martin-luther-kings-i-have-a-dream-speech-august-28-1963.php']

import requests
import json

insightIP = 'http://178.62.229.16'
insightPort = '8484'
insightVersion = 'v1.0'
apikey = 'thisisinsightapikey'

# parameters of news recommendation are twitter_ids, crm_ids
"""
twitter_ids = []
crm_ids = [135574293]
insightSetting = insightIP + ':' + insightPort + '/api/' + insightVersion 
request = '/recommendation/news?' + 'twitter_ids=' + str(twitter_ids) + '&' + 'crm_ids=' + str(crm_ids)
"""

TXT = []

for url in urls:
    insightSetting = insightIP + ':' + insightPort + '/api/' + insightVersion
    request = '/text_analytics/url_scraper?api_key='+apikey+'&url='+url
    #res = requests.get(insightSetting + request)
    print(insightSetting + request)
    res = requests.get(insightSetting + request)
    data = res.json()
    TXT.append(data['text'])

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import pandas as pd

import os

import nltk
from utils import tokenizer
import nltk
from nltk import FreqDist
from nltk.stem.porter import PorterStemmer
from numpy import log, mean
import json, csv, re
import pprint as pp


path = '/Users/cemgil/src/OpenMaker/Semantics/output/'
fn = os.listdir(path)

DF = []
CATEGORIES = []
for f in fn:
    print(f)
    df = pd.read_csv(path+f)
    DF.append(df)
    #CATEGORIES.append(f.split('_')[1].split('.')[0])
    CATEGORIES.append(f[9:].split('.')[0])

    
catalog = []
for i in range(len(DF)):
    catalog.append({u[1].Stem: u[1].Score for u in DF[i].iterrows()})

In [ ]:
def Text2Score(txt, catalog):

    with open("data/stopwords_standard.txt", "r") as f:
        STOP_WORDS_STANDARD = set(f.read().strip().split("\n"))
    #print(STOP_WORDS_STANDARD)

    with open("data/stopwords_openmaker.txt", "r") as f:
        STOP_WORDS_OPENMAKER = set(f.read().strip().split("\n"))
    #print(STOP_WORDS_OPENMAKER)

    # merging the two list together
    STOP_WORDS = STOP_WORDS_STANDARD.union(STOP_WORDS_OPENMAKER)

    tokens = nltk.word_tokenize(txt)
    token_counts = FreqDist(tokens)
    tokenizer.CHARACTERS_TO_SPLIT += '‘'+'’'+'“'+'”'+'.'

    for stopword in STOP_WORDS:
        if stopword in token_counts:
            del token_counts[stopword]

    for punctuation in tokenizer.CHARACTERS_TO_SPLIT:
        if punctuation in token_counts:
            del token_counts[punctuation]


    pattern_letters = re.compile('[a-z]')
    def has_letters(x):
        return(pattern_letters.search(x) is not None)

    reduced = {k:v for k,v in token_counts.items() if has_letters(k)}
    #print("Reduction due to all number matches: ", len(token_counts) - len(reduced))
    token_counts = reduced

    reduced = {k:v for k,v in token_counts.items() if len(k) > 1}
    #print("Reduction due to single characters: ", len(token_counts) - len(reduced))
    token_counts = reduced

    stemmer = PorterStemmer()
    input_wset_stems = {k: stemmer.stem(k) for k in token_counts.keys()}

    Scores = []
    for cat_idx in range(len(fn)):
        sum_score = 0
        notin_catalog = 0
        for p in token_counts.keys():
            if p in input_wset_stems.keys():
                r = input_wset_stems[p]
            else:
                r = p

            if p in catalog[cat_idx].keys():
                #sum_score += catalog[cat_idx][p] 
                #print(catalog[cat_idx][p], p,token_counts[p])
                None
            else:
                #print('No Score', p,token_counts[p])
                #notin_catalog+=1
                None

            if r in catalog[cat_idx].keys():
                sum_score += catalog[cat_idx][r] 
            else:
                notin_catalog+=1

        Scores.append(sum_score)

    return Scores

In [ ]:
for txt in TXT:
    Score = Text2Score(txt, catalog)
    categories_desired_order = [4,0,3,7,6,9,1,8,2,5]

    CatScore = [(CATEGORIES[i], Score[i]-min(Score)) for i in categories_desired_order]    
    print(CatScore)
    RadarPlot(CatScore)

In [ ]:
# Plots a radar chart.

from math import pi
import matplotlib.pyplot as plt

def RadarPlot(CatScore):
    # Set data
    cat = [a[0] for a in CatScore]
    values = [np.abs(a[1]) for a in CatScore]

    N = len(cat)

    x_as = [n / float(N) * 2 * pi for n in range(N)]

    # Because our chart will be circular we need to append a copy of the first 
    # value of each list at the end of each list with data
    values += values[:1]
    x_as += x_as[:1]


    # Set color of axes
    plt.rc('axes', linewidth=0.5, edgecolor="#888888")


    # Create polar plot
    ax = plt.subplot(111, polar=True)


    # Set clockwise rotation. That is:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)


    # Set position of y-labels
    ax.set_rlabel_position(0)


    # Set color and linestyle of grid
    ax.xaxis.grid(True, color="#888888", linestyle='solid', linewidth=0.5)
    ax.yaxis.grid(True, color="#888888", linestyle='solid', linewidth=0.5)


    # Set number of radial axes and remove labels
    plt.xticks(x_as[:-1], [])

    # Set yticks
    plt.yticks([],[])


    # Plot data
    ax.plot(x_as, values, linewidth=0, linestyle='solid', zorder=3)

    # Fill area
    ax.fill(x_as, values, 'b', alpha=0.3)


    # Set axes limits
    plt.ylim(0, max(values))


    # Draw ytick labels to make sure they fit properly
    for i in range(N):
        angle_rad = i / float(N) * 2 * pi

        if angle_rad == 0:
            ha, distance_ax = "center", 10
        elif 0 < angle_rad < pi:
            ha, distance_ax = "left", 1
        elif angle_rad == pi:
            ha, distance_ax = "center", 1
        else:
            ha, distance_ax = "right", 1

        ax.text(angle_rad, max(values) + distance_ax, cat[i], size=10, horizontalalignment=ha, verticalalignment="center")


    # Show polar plot
    plt.show()